In [7]:
import json
import jsonlines

### Create Persona-Triple Aligned dictionary (DNLI)

In [8]:
data = {} #{sentence: triple_set}, triple_set = {s1-r1-s2}
relations = set()
head = set()

for t_type in ['train', 'dev', 'test']:
    with jsonlines.open(f"data/dnli/dialogue_nli/dialogue_nli_{t_type}.jsonl") as file:
        for instances in file.iter():
            for instance in instances:
                s1, s2 = instance['sentence1'], instance['sentence2']
                t1, t2 = instance['triple1'], instance['triple2']
                for sent in [(s1,t1), (s2,t2)]:
                    if sent[0] not in data.keys():
                        data[sent[0]] = set()
                    data[sent[0]].add(f"{sent[1][0]}\t{sent[1][1]}\t{sent[1][2]}")
                    relations.add(sent[1][1])
                    head.add(sent[1][0])

In [9]:
print(len(head))
head

181


{'<blank>',
 '<none>',
 'a already',
 'a film',
 'abby',
 'ariel',
 'best friend',
 'both parents',
 'boyfriend',
 'brothers and sisters',
 'buying',
 'calden',
 'channing tatum',
 'child',
 'chocolate chewy baked squares',
 'chocolate chip cookies',
 'clothing',
 'commitment',
 'connor mcgreggor',
 'customer',
 'dad and mom',
 'date',
 'daughter',
 'dear ones',
 'dog',
 'dude',
 'everyone',
 'family',
 'farming',
 'farms',
 'felines',
 'females from other countries',
 'ferrari',
 'folks',
 'foreign cars',
 'fried catfish',
 'friend',
 'friends',
 'friends and i',
 'girlfriend',
 'god',
 'grandma',
 'grandmother',
 'he',
 'hiking and running',
 'home grown vegetables',
 'hot choice',
 'husband',
 'i',
 'i am',
 'in law',
 'it',
 "it's",
 'italian food',
 'jordan',
 'karate and taekwondo',
 'kid',
 'kitten fluffy',
 'krispy kreme',
 'me',
 'milo',
 'mom and dad',
 'my',
 'my adopted father',
 'my band',
 'my best buddy',
 'my best friend',
 'my bestfriend',
 'my body',
 'my book',
 'my 

### Assign Persona Sentences Triples (Align DNLI-PC)

In [10]:
pc = json.load(open('data/pc_dataset/personachat_self_original.json', 'r'))

In [11]:
for f_type in ['train', 'valid']:
    print(pc.keys())
    d_type = pc[f_type]
    new_data = []
    out_data = jsonlines.open(f"data/preprocessed/raw/{f_type}.jsonl", "w")
    fail = 0
    for dialogue in d_type:
        personas = dialogue['personality']
        utterances = dialogue['utterances']
        triples = set()
        iGraph = False
        for p in personas:
            if p in data.keys():
                for trip in data[p]:
                    if trip.split("\t")[0]=="i":
                        iGraph = True
                    triples.add(trip)
        if iGraph:
            for utt in utterances:
                query = utt['history'][-1]
                resp = utt['candidates'][-1]
                new_data.append({"triples":list(triples), "query":query, "response":resp})
        else: fail+=1
    new_data = list(new_data)
    for instance in new_data[:100]:
        out_data.write(instance)
    
    print(fail, len(d_type), fail/len(d_type))

dict_keys(['train', 'valid'])
43 17878 0.0024051907372189285
dict_keys(['train', 'valid'])
2 1000 0.002


In [12]:
for d_type in [train, dev]:
    print("---------")
    count=0
    total = 0
    d_total = 0
    d_cnt = 0
    for dialogue in d_type:
        personas = dialogue['personality']
        utterances = dialogue['utterances']
        d_fail = False
        for p in personas:
            if p not in data.keys():
                count+=1
                d_fail = True
            total+=1
        d_total +=1
        if d_fail: d_cnt+=1
    
    print(count, total, count/total)
    print(d_cnt, d_total, d_cnt/d_total)

NameError: name 'train' is not defined